In [1]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # Import the exception
import time
import base64
import os

In [2]:
headers = {'User-Agent': 'Mozilla/5.0'}

def fetch_page(url):
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.text, 'html.parser')

def find_player_links(team_url):
    soup = fetch_page(team_url)
    player_links = set()  # Use a set to automatically remove duplicates
    for a_tag in soup.find_all('a', href=True):
        if re.match(r'/player/.+/\d+', a_tag['href']):
            full_url = 'https://www.sofascore.com' + a_tag['href']
            player_links.add(full_url)
    return list(player_links)  # Convert back to list if needed

# List of team URLs
team_urls = [
    'https://www.sofascore.com/team/football/arsenal/42',
    'https://www.sofascore.com/team/football/liverpool/44',
    'https://www.sofascore.com/team/football/manchester-city/17',
    'https://www.sofascore.com/team/football/aston-villa/40',
    'https://www.sofascore.com/team/football/tottenham-hotspur/33',
    'https://www.sofascore.com/team/football/manchester-united/35',
    'https://www.sofascore.com/team/football/west-ham-united/37',
    'https://www.sofascore.com/team/football/brighton-and-hove-albion/30',
    'https://www.sofascore.com/team/football/wolverhampton/3',
    'https://www.sofascore.com/team/football/newcastle-united/39',
    'https://www.sofascore.com/team/football/chelsea/38',
    'https://www.sofascore.com/team/football/fulham/43',
    'https://www.sofascore.com/team/football/bournemouth/60',
    'https://www.sofascore.com/team/football/crystal-palace/7',
    'https://www.sofascore.com/team/football/brentford/50',
    'https://www.sofascore.com/team/football/everton/48',
    'https://www.sofascore.com/team/football/luton-town/72',
    'https://www.sofascore.com/team/football/nottingham-forest/14',
    'https://www.sofascore.com/team/football/burnley/6',
    'https://www.sofascore.com/team/football/sheffield-united/15',
    'https://www.sofascore.com/team/football/real-madrid/2829',
    'https://www.sofascore.com/team/football/barcelona/2817',
    'https://www.sofascore.com/team/football/girona-fc/24264',
    'https://www.sofascore.com/team/football/athletic-club/2825',
    'https://www.sofascore.com/team/football/atletico-madrid/2836',
    'https://www.sofascore.com/team/football/real-sociedad/2824',
    'https://www.sofascore.com/team/football/real-betis/2816',
    'https://www.sofascore.com/team/football/valencia/2828',
    'https://www.sofascore.com/team/football/villarreal/2819',
    'https://www.sofascore.com/team/football/getafe/2859',
    'https://www.sofascore.com/team/football/las-palmas/6577',
    'https://www.sofascore.com/team/football/osasuna/2820',
    'https://www.sofascore.com/team/football/deportivo-alaves/2885',
    'https://www.sofascore.com/team/football/mallorca/2826',
    'https://www.sofascore.com/team/football/rayo-vallecano/2818',
    'https://www.sofascore.com/team/football/sevilla/2833',
    'https://www.sofascore.com/team/football/celta-vigo/2821',
    'https://www.sofascore.com/team/football/cadiz/4488',
    'https://www.sofascore.com/team/football/granada/33779',
    'https://www.sofascore.com/team/football/almeria/2858',
    'https://www.sofascore.com/team/football/bayer-04-leverkusen/2681',
    'https://www.sofascore.com/team/football/fc-bayern-munchen/2672',
    'https://www.sofascore.com/team/football/vfb-stuttgart/2677',
    'https://www.sofascore.com/team/football/borussia-dortmund/2673',
    'https://www.sofascore.com/team/football/rb-leipzig/36360',
    'https://www.sofascore.com/team/football/eintracht-frankfurt/2674',
    'https://www.sofascore.com/team/football/fc-augsburg/2600',
    'https://www.sofascore.com/team/football/tsg-hoffenheim/2569',
    'https://www.sofascore.com/team/football/sc-freiburg/2538',
    'https://www.sofascore.com/team/football/sv-werder-bremen/2534',
    'https://www.sofascore.com/team/football/1-fc-heidenheim/5885',
    'https://www.sofascore.com/team/football/borussia-mgladbach/2527',
    'https://www.sofascore.com/team/football/1-fc-union-berlin/2547',
    'https://www.sofascore.com/team/football/vfl-wolfsburg/2524',
    'https://www.sofascore.com/team/football/vfl-bochum-1848/2542',
    'https://www.sofascore.com/team/football/1-fsv-mainz-05/2556',
    'https://www.sofascore.com/team/football/1-fc-koln/2671',
    'https://www.sofascore.com/team/football/darmstadt-98/2576',
    'https://www.sofascore.com/team/football/inter/2697',
    'https://www.sofascore.com/team/football/milan/2692',
    'https://www.sofascore.com/team/football/juventus/2687',
    'https://www.sofascore.com/team/football/bologna/2685',
    'https://www.sofascore.com/team/football/roma/2702',
    'https://www.sofascore.com/team/football/atalanta/2686',
    'https://www.sofascore.com/team/football/napoli/2714',
    'https://www.sofascore.com/team/football/fiorentina/2693',
    'https://www.sofascore.com/team/football/lazio/2699',
    'https://www.sofascore.com/team/football/monza/2729',
    'https://www.sofascore.com/team/football/torino/2696',
    'https://www.sofascore.com/team/football/genoa/2713',
    'https://www.sofascore.com/team/football/lecce/2689',
    'https://www.sofascore.com/team/football/udinese/2695',
    'https://www.sofascore.com/team/football/hellas-verona/2701',
    'https://www.sofascore.com/team/football/cagliari/2719',
    'https://www.sofascore.com/team/football/empoli/2705',
    'https://www.sofascore.com/team/football/frosinone/2801',
    'https://www.sofascore.com/team/football/sassuolo/2793',
    'https://www.sofascore.com/team/football/salernitana/2710',
    'https://www.sofascore.com/team/football/paris-saint-germain/1644',
    'https://www.sofascore.com/team/football/stade-brestois/1715',
    'https://www.sofascore.com/team/football/as-monaco/1653',
    'https://www.sofascore.com/team/football/lille/1643',
    'https://www.sofascore.com/team/football/nice/1661',
    'https://www.sofascore.com/team/football/lens/1648',
    'https://www.sofascore.com/team/football/olympique-de-marseille/1641',
    'https://www.sofascore.com/team/football/stade-rennais/1658',
    'https://www.sofascore.com/team/football/stade-de-reims/1682',
    'https://www.sofascore.com/team/football/olympique-lyonnais/1649',
    'https://www.sofascore.com/team/football/toulouse/1681',
    'https://www.sofascore.com/team/football/rc-strasbourg/1659',
    'https://www.sofascore.com/team/football/le-havre/1662',
    'https://www.sofascore.com/team/football/montpellier/1642',
    'https://www.sofascore.com/team/football/lorient/1656',
    'https://www.sofascore.com/team/football/nantes/1647',
    'https://www.sofascore.com/team/football/metz/1651',
    'https://www.sofascore.com/team/football/clermont-foot/1680',
        # Add more team URLs here
]

# Collect player URLs for each team
all_player_urls = set()  # Use a set to avoid duplicates across teams
for team_url in team_urls:
    player_urls = find_player_links(team_url)
    all_player_urls.update(player_urls)  # Add player URLs from this team to the overall set

# Convert back to list if needed and print
all_player_urls = list(all_player_urls)
for url in all_player_urls:
    print(url)

https://www.sofascore.com/player/isak-vural/1210995
https://www.sofascore.com/player/buyukarslan-oguzcan/1413584
https://www.sofascore.com/player/emanuel-emegha/1048333
https://www.sofascore.com/player/stefano-sensi/294313
https://www.sofascore.com/player/darren-randolph/13060
https://www.sofascore.com/player/moise-kean/835601
https://www.sofascore.com/player/diego-rico/350560
https://www.sofascore.com/player/marvelous-nakamba/556904
https://www.sofascore.com/player/darko-lazovic/41157
https://www.sofascore.com/player/pierre-emile-hojbjerg/149593
https://www.sofascore.com/player/mathys-tel/1048888
https://www.sofascore.com/player/chris-fuhrich/891510
https://www.sofascore.com/player/adrien-tameze/139235
https://www.sofascore.com/player/corazza-tommaso/1137366
https://www.sofascore.com/player/arouna-sangante/1035988
https://www.sofascore.com/player/mattia-zaccagni/293581
https://www.sofascore.com/player/andoni-gorosabel/866810
https://www.sofascore.com/player/fabian-nurnberger/940886
ht

In [3]:
# Construct the path to chromedriver
current_dir = os.getcwd()  # Get the current working directory
chrome_driver_path = os.path.join(current_dir, "chromedriver-win64", "chromedriver.exe")

In [4]:
# Setup Selenium WebDriver
service = Service(executable_path=os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe"))
driver = webdriver.Chrome(service=service)
driver.fullscreen_window()  # Make the browser window fullscreen

def save_player_position(player_url):
    player_name = player_url.split('/')[-2].replace('-', '_')
    try:
        driver.get(player_url)
        time.sleep(5)  # Adjust time as needed for the page to fully load
        
        # JavaScript script to extract position data from SVG elements
        js_script = """
        var positions = [];
        var elements = document.querySelectorAll('g.sc-f11cf694-0.gWtzRI text');
        elements.forEach(function(el) {
            positions.push(el.textContent.trim());
        });
        return positions;
        """
        
        # Execute the JavaScript to get the positions
        positions = driver.execute_script(js_script)
        
        # Save the extracted positions to a text file if any are found
        if positions:
            with open(f'{player_name}_positions.txt', 'w') as file:
                for position in positions:
                    file.write(f'{position}\n')
        else:
            print(f'No position data found for {player_name}.')
                
    except NoSuchElementException:
        print(f'Position data not found for {player_name}. Skipping...')

# Use the all_player_urls list to download heatmap images
for url in all_player_urls:
    save_player_position(url)

driver.quit()

No position data found for isak_vural.
No position data found for buyukarslan_oguzcan.
No position data found for wharton_adam.
No position data found for luke_knapp.
No position data found for diop_edan.
No position data found for sane_ibou.
No position data found for shandon_baptiste.
No position data found for harpreet_ghotra.
No position data found for marcel_lubik.
No position data found for stefan_schimmer.
No position data found for ryan_don_naderi.
No position data found for emanuel_vignato.
No position data found for kaua_dos_santos.
No position data found for cartillier_antonin.
No position data found for henry_blackledge.
No position data found for rasoulinia_nick.
No position data found for kevin_schade.
No position data found for axel_camblan.
No position data found for alessio_zerbin.
No position data found for gaetano_gianluca.
No position data found for schleinitz_tim.
No position data found for aourir_ayman.
No position data found for jerome_scholz.
No position data fo